In [2]:
# all imports
import numpy as np
import os
import requests
from astropy.table import Table, vstack
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.wcs import WCS
import math
import matplotlib as mpl
from astropy.visualization.wcsaxes import SphericalCircle
import pandas as pd

In [3]:
# define directories
data_dir = '/global/cfs/projectdirs/desi/science/td/pv/tfgalaxies/'
cache_dir = '/pscratch/sd/j/jjpim/cache/' # my cache path
vi_dir = '/pscratch/sd/j/jjpim/visual_inspection/' # vi image path

In [4]:
# load updated fuji table
tf_fuji = Table.read(data_dir + 'SV/desi_pv_tf_fuji_healpix_rotcurve_EOA_VI.fits') 
tf_fuji[:5]

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC,OBS_IN_SV,SKY_FIBER_DIST,SKY_FIBER_DIST_R26,ROT_CURVE,EOA
int64,float64,float64,int64,bytes3,float64,float64,int64,float64,bytes63,bytes3,int64,float64,float64,float64,float64,float64,float64,float64
1079550234591232,194.390863195343,27.5157211790145,10378,sv3,1.1235686466514,7.31685779475115e-05,4,3.28414569795132,fuji/healpix/sv3/bright/103/10378/redrock-sv3-bright-10378.fits,EXT,662902,194.39086319534337,27.51572117901454,1.0,0.03927420444753712,1.0000000170652443,0.0,0.0
1092744374124544,194.390863195343,27.5157211790145,10378,sv3,0.686773088332363,6.9756676262104e-05,4,0.786607094109058,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.39086319534337,27.51572117901454,1.0,0.03927420444753712,1.0000000170652443,0.0,0.0
1092744374124546,194.364461113654,27.5037185881314,10378,sv3,0.0242933923052181,4.95233472646785e-05,0,95.428411073226,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.36446111365385,27.50371858813136,1.0,0.012960487467698754,0.3300000056318236,0.0,0.0
1092744369930240,194.338458724402,27.4918902690326,10378,sv3,0.0264170223697961,0.00010139452689994,0,9.53278421035066,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.33845872440244,27.491890269032595,1.0,0.012960487467685656,0.33000000563149007,0.0,0.0
1092744374124545,194.377858465028,27.5098100780282,10378,sv3,0.211332646769145,6.68535116703737e-05,4,3.73989077657461,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.3778584650283,27.509810078028195,1.0,0.026313716979861984,0.6700000114340222,0.0,0.0


In [5]:
# load iron table
tf_iron = Table.read(data_dir + 'Y1/desi_pv_tf_iron_healpix_rotcurve_EOA_VI.fits')
tf_iron[:5]

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC,OBS_IN_SV,SKY_FIBER_DIST,SKY_FIBER_DIST_R26,ROT_CURVE,EOA
int64,float64,float64,int64,bytes4,float64,float64,int64,float64,bytes65,bytes3,int64,float64,float64,float64,float64,float64,float64,float64
2852147603439621,198.369130660983,36.5372037049171,10475,main,0.815976335547845,7.38513168100107e-05,4,0.128754377365112,iron/healpix/main/dark/104/10475/redrock-main-dark-10475.fits,EXT,649377,198.36913066098333,36.537203704917076,0.0,0.0567420462925582,0.599999979158402,0.0,1.0
2399148812795907,198.371733180003,36.4994335406917,10475,main,1.11088784970434,7.48767797671894e-05,4,7.9473560154438,iron/healpix/main/bright/104/10475/redrock-main-bright-10475.fits,EXT,649377,198.37173318000336,36.499433540691676,0.0,0.09457007715465947,0.9999999652681886,0.0,1.0
2399382443917318,184.845242475328,49.8157304793777,10995,main,1.14739342108157,0.000146302276719084,4,2.56771463155746,iron/healpix/main/bright/109/10995/redrock-main-bright-10995.fits,EXT,1008911,184.84524247532795,49.81573047937771,0.0,0.024841917763806447,0.9999999684347345,0.0,0.0
2399634072797192,184.341289722203,70.8283725474297,11965,main,1.51703376230705,6.28979649962091e-05,4,4.76254060305655,iron/healpix/main/bright/119/11965/redrock-main-bright-11965.fits,EXT,241234,184.34128972220284,70.82837254742968,0.0,0.026555350833845817,0.8000000542794348,0.0,0.0
2852141710442505,123.256011148025,36.2652948002806,6448,main,0.00787379494184006,3.4714052819995e-05,0,22.1719104201402,iron/healpix/main/dark/64/6448/redrock-main-dark-6448.fits,EXT,31591,123.25601114802525,36.26529480028061,0.0,0.015878717350385107,0.40000000619171416,0.0,0.0


In [6]:
tf_fuji['VELOCITY']=np.nan
tf_fuji['VELOCITY_REL']=np.nan
tf_iron['VELOCITY']=np.nan
tf_iron['VELOCITY_REL']=np.nan

In [23]:
# Filter fuji and iron for viable targets
filtered_tf_fuji = tf_fuji[tf_fuji['SKY_FIBER_DIST_R26'] < 0.001]
filtered_tf_iron = tf_iron[tf_iron['SKY_FIBER_DIST_R26'] < 0.001]
filtered_tf_fuji = filtered_tf_fuji[filtered_tf_fuji['ROT_CURVE'] == 1]
filtered_tf_iron = filtered_tf_iron[filtered_tf_iron['ROT_CURVE'] == 1]

# Find and print the maximum Z value
max_value_fuji = filtered_tf_fuji['Z'].max()
max_value_iron = filtered_tf_iron['Z'].max()
print("Maximum value in tf_fuji:", max_value_fuji)
print("Maximum value in tf_iron:", max_value_iron)

Maximum value in tf_fuji: 0.173630148670864
Maximum value in tf_iron: 1.62543750380985


In [24]:
#pull out all unique SGA ID's for iron and fuji
tf_fuji_rot = np.unique(filtered_tf_fuji[filtered_tf_fuji['ROT_CURVE']==1]['SGA_ID'])
tf_iron_rot = np.unique(filtered_tf_iron[filtered_tf_iron['ROT_CURVE']==1]['SGA_ID'])
#combine iron and fuji unique ID's, print final list 
tf_galaxies = np.unique(np.concatenate((tf_fuji_rot, tf_iron_rot)))
tf_galaxies = tf_galaxies.tolist()
print(tf_galaxies)

[123, 896, 1548, 1583, 1980, 2497, 5732, 5958, 5964, 6443, 8032, 9338, 9519, 9769, 10133, 10325, 10553, 11977, 11998, 12585, 12892, 13851, 13880, 14501, 15005, 16256, 16383, 18191, 19235, 19278, 19682, 20179, 20258, 20886, 21905, 23897, 25100, 25532, 25835, 26866, 26930, 28423, 28642, 29121, 29289, 30149, 31127, 31220, 32406, 32484, 32907, 33662, 34497, 35184, 35712, 36532, 36904, 36913, 37543, 39247, 39919, 42992, 43211, 46327, 50360, 54974, 56757, 57891, 58617, 61908, 71578, 73111, 79898, 81194, 81587, 88963, 90791, 91660, 92403, 92536, 93731, 98148, 98921, 98934, 100987, 102737, 103508, 103535, 104913, 105847, 106536, 107105, 108576, 108893, 109333, 110814, 111102, 113767, 114405, 114489, 115283, 116764, 117242, 117558, 118397, 122260, 122580, 122810, 124900, 125245, 125611, 126630, 127141, 127268, 128130, 128163, 128411, 128944, 129388, 130437, 130766, 133855, 135061, 135235, 136307, 139660, 149009, 149502, 153504, 154215, 156492, 158556, 158910, 159963, 166290, 170351, 171058, 171

In [69]:
# find all target ID's for one galaxy
targets_fuji=tf_fuji[tf_fuji['SGA_ID']==  123]
targets_iron=tf_iron[tf_iron['SGA_ID']== 123]
target_list=(vstack([targets_fuji,targets_iron]))
print(target_list)

     TARGETID        TARGET_RA        TARGET_DEC    ... VELOCITY VELOCITY_REL
----------------- ---------------- ---------------- ... -------- ------------
 1071233026228227 242.564878755415 53.6763270782975 ...      nan          nan
 1071233026228228 242.564908359756 53.6746327647682 ...      nan          nan
 1071233026228231 242.565028546859 53.6677528249076 ...      nan          nan
 1084427165761544 242.565058139153 53.6660585113414 ...      nan          nan
39633304835591852 242.564968458214  53.671192794853 ...      nan          nan
39633304835591852 242.564968458214  53.671192794853 ...      nan          nan


In [70]:
#print target Z value inofrmation
z_targets=(target_list['Z'])
print(z_targets)

        Z        
-----------------
0.184971788085982
  1.5161235823035
0.223582059648363
0.125989095083328
0.109965464429358
0.109966732247689


In [50]:
#list of good centers in fuji for one galaxy found by SGA ID above
fuji_centers = targets_fuji[targets_fuji['SKY_FIBER_DIST_R26'] < 0.001]
fuji_centers = fuji_centers[fuji_centers['ZWARN']==0]
fuji_good_centers = fuji_centers[fuji_centers['DELTACHI2']>25] 
fuji_good_centers

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC,OBS_IN_SV,SKY_FIBER_DIST,SKY_FIBER_DIST_R26,ROT_CURVE,EOA,VELOCITY,VELOCITY_REL
int64,float64,float64,int64,bytes3,float64,float64,int64,float64,bytes63,bytes3,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64
39633304835591852,242.564968458214,53.671192794853,10153,sv3,0.109965464429358,1.30236298072821e-05,0,847.705579817295,fuji/healpix/sv3/bright/101/10153/redrock-sv3-bright-10153.fits,TFT,123,242.56496845821405,53.67119279485298,1.0,1.6837541040866038e-14,3.2792580096330368e-12,1.0,0.0,nan,nan


In [91]:
#list of good centers in iron for one galaxy found by SGA ID above
iron_centers = targets_iron[targets_iron['SKY_FIBER_DIST_R26'] < 0.001]
iron_centers = iron_centers[iron_centers['ZWARN']==0]
iron_good_centers = iron_centers[iron_centers['DELTACHI2']>25] 
iron_good_centers

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC,OBS_IN_SV,SKY_FIBER_DIST,SKY_FIBER_DIST_R26,ROT_CURVE,EOA,VELOCITY,VELOCITY_REL
int64,float64,float64,int64,bytes4,float64,float64,int64,float64,bytes65,bytes3,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64
39633304835591852,242.564968458214,53.671192794853,10153,sv3,0.109966732247689,1.26106529808024e-05,0,879.694150358438,iron/healpix/sv3/bright/101/10153/redrock-sv3-bright-10153.fits,TFT,123,242.56496845821405,53.67119279485298,1.0,1.6837541040866038e-14,3.2792580096330368e-12,1.0,0.0,nan,nan


In [96]:
#need to test with a galaxy that has more than 2 center targets
#defining terms
z_fuji = fuji_good_centers['Z']
z_iron = iron_good_centers['Z']
z_fuji_err = fuji_good_centers['ZERR']
z_iron_err = iron_good_centers['ZERR']

# calculating the weights
w_fuji = 1 / z_fuji_err
w_iron = 1 / z_iron_err

# calculating the weighted average z naught
w1 = np.sum(w_fuji * z_fuji)
w2 = np.sum(w_iron * z_iron)
wn = np.sum(w_fuji + w_iron)
z_o = (w1 + w2) / wn

print(z_o)

#define z naught error
#might need to do np.sum on these too? 
z_o_err = (1 / (w_1 + w_2))*(((w_1 * z_1_err)**2 + (w_2 * z_2_err))**0.5)
z_o_err = (1/ wn) * (((w_1 * z_1_err)**2 + (w_2 * z_2_err)**2)**0.5)
print(z_o_err)

0.10996610855101113
         ZERR        
---------------------
9.060735335818906e-06


In [93]:
#define z error for all targets
for TARGETID in target_list:
        z_err=target_list['ZERR']
print(z_err)

        ZERR        
--------------------
4.36696398757541e-05
9.98100168252869e-05
 6.6695997421467e-05
 3.5227684897735e-05
1.30236298072821e-05
1.26106529808024e-05


In [94]:
#find delta z from z naught and z values of all viable targets 
delta_z=z_targets - z_o
print(delta_z)

          Z           
----------------------
   0.07500567953497086
     1.406157473752489
   0.11361595109735187
  0.016022986532316874
-6.441216531261063e-07
   6.2369667787332e-07


In [95]:
#use delta z to find velocity
c = 299792.458
velocity = c*delta_z
print(velocity)
#results seem too big, z has no units and c is in km/s so it should be right but have nitya double check units
#put results into new column on tables

         Z          
--------------------
   22486.13703174921
   421555.4053913291
   34061.20524748291
   4803.570517024172
-0.19310281364169876
  0.1869795601060768


find v error for each target

In [ ]:
#find v error for each target
for i,v in z_o_err:
    z_o_error[i]=v**2
    print(z_o_error)+
#v_err=c*[math.sqrt([[z_o_err]**2]+[[z_err]**2])]
# put v error into new column for each table